<img align="right" src="tf-small.png"/>

# Atoms and Mothers

One of the trickiest bits in the 
[ETCBC database of the Hebrew Bible](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_home.html)
are the
[*atoms*](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/otype.html#linguistic-types)
within sentences, clauses and phrases, and the
[*mother*](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/mother.html)
relationship between objects.

Yet a lot of the coding effort of the ETCBC is located in precisely these concepts, especially in the treatment of *clause*-atoms.
For example, there is a specific feature
[code](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/code.html)
defined on clause atoms that provides a refined categorization of clauses.

In this notebook, we will explore and highlight what you can do with mothers and clause_atoms.

# Acknowledgement

This notebook owes a lot to the eager questions of Joshua Grauman and the prompt answers by Hendrik-Jan Bosman, spiced with additional insights of Cody Kingham and David van Acker.

In [22]:
import collections
from tf.fabric import Fabric

In [ ]:
ETCBC = 'hebrew/etcbc4c'
TF = Fabric( modules=ETCBC )

In [3]:
api = TF.load('''
    typ
''')
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.31s B typ                  from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s Feature overview: 102 for nodes; 5 for edges; 1 configs; 7 computed
  0.36s All features loaded/computed - for details use loadLog()


# clause and clause_atom

The ETCBC does not work with *embedded* clauses. In the clause

`we'll see whether this works out later`

there is an inner clause `whether this works out`, and an outer clause `we'll see ... later`.

In many types of linguistic analysis, the inner clause is part of the outer clause, in the role of
direct object. The word `works` belongs both to the inner and outer clause.

Not so in the ETCBC analysis of things. 
The inner clause *interrupts* the outer clause, and the outer clause has a *gap*.
The word `works` belongs to the inner clause only.

Because of the gap, the outer clause splits into two segments, one before the gap, and one after the gap.
These parts are called the *clause_atoms*.

The clause_atom before the gap is rather complete, it has a subject and a predicate.
The clause_atom after the gap is, well, defective.

## Explore

Let us see some clauses that consist of multiple clause atoms.

In [45]:
results = list(S.search('''
clause
  clause_atom
  < clause_atom
'''))

mClauses = sortNodes(set(x[0] for x in results))
info('{} multiple atom clauses'.format(len(mClauses)))

  0.68s 2441 multiple atom clauses


In [46]:
for r in results[0:5]: print(S.glean(r))

clause[וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ ...] clause_atom[וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ ] clause_atom[וּבֵ֣ין הַמַּ֔יִם ]
clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause_atom[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ] clause_atom[עֵ֣ץ פְּרִ֞י ]
clause[עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו עַל־...] clause_atom[עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו ] clause_atom[עַל־הָאָ֑רֶץ ]
clause[וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא ...] clause_atom[וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא ...] clause_atom[וְעֵ֧ץ ]
clause[עֹ֥שֶׂה פְּרִ֛י לְמִינֵ֑הוּ ] clause_atom[עֹ֥שֶׂה פְּרִ֛י ] clause_atom[לְמִינֵ֑הוּ ]


Now count the how many clauses have how many atoms.

In [47]:
caCount = collections.Counter()
for c in mClauses:
    caCount[len(L.d(c, 'clause_atom'))] += 1
for (nca, nc) in sorted(caCount.items(), key=lambda x: (-x[1], x[0])):
    print('{:>2} atoms: {:>5} clauses'.format(nca, nc))

 2 atoms:  2334 clauses
 3 atoms:    95 clauses
 4 atoms:    10 clauses
 5 atoms:     2 clauses


The next thing is: we want to see every multi-atom clause, and for each atom at which slot it starts and end, and whether its 
[clause type](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/typ.html)
is defective or not.

In [50]:
chunks = []
for c in mClauses:
    cas = L.d(c, 'clause_atom')
    cwords = L.d(c, otype='word')
    rep = ['{}-{}'.format(cwords[0], cwords[-1])]
    for ca in cas:
        defc = F.typ.v(ca) == 'Defc'
        slots = L.d(ca, otype='word')
        bs = slots[0]
        es = slots[-1]
        rep.append('\t{}-{}-{}'.format(bs, 'D' if defc else '-', es))
    chunks.append(rep)

for ch in chunks[0:10]: print('\n'.join(ch))

101-115
	101---105
	112-D-115
182-190
	182---186
	189-D-190
191-200
	191---194
	198-D-200
204-215
	204---209
	214-D-215
216-222
	216---217
	221-D-222
380-401
	380---391
	400-D-401
593-611
	593---598
	607-D-611
622-650
	622---640
	645-D-650
989-995
	989-D-991
	994---995
1097-1110
	1097---1103
	1109-D-1110


Is it the case that every clause splits into exactly one non-defective atom and the rest defective?
Lets count the profiles of clauses. A profile is a sequence of `-` and `D` characters, corresponding to the
defectiveness of its successive clause_atoms.

In [55]:
profiles = collections.Counter()
for c in F.otype.s('clause'):
    cas = L.d(c, 'clause_atom')
    profile = ''.join('D' if F.typ.v(ca) == 'Defc' else '-' for ca in L.d(c, otype='clause_atom'))
    profiles[profile] += 1
info('{} profiles'.format(len(profiles)))
for (profile, n) in sorted(profiles.items()):
    print('{:<6} : {:>5}x'.format(profile, n))

13m 01s 10 profiles
-      : 85559x
-D     :  1083x
-DD    :    62x
-DDD   :     7x
-DDDD  :     2x
D-     :  1251x
D-D    :    10x
DD-    :    23x
DD-D   :     1x
DDD-   :     2x


This gives a pretty good picture of the construction of clauses out of their atoms.
Note that we have inspected all clauses, including the single atoms clauses, and note that those are never
defective.